In [13]:
#define the class

import torch
import torch.nn as nn 
import torch.nn.functional as F
import numpy as np

class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1=nn.Conv2d(1,10,3)
        self.pool=nn.MaxPool2d(2,2)
        self.conv2=nn.Conv2d(10,16,3)
        self.fc1=nn.Linear(16*10*10, 120)
        self.fc2=nn.Linear(120, 84)
        self.fc3=nn.Linear(84, 2)
        
    def forward(self,x):
        x=self.pool(F.relu(self.conv1(x)))
        x=self.pool(F.relu(self.conv2(x)))
        #print(x.size())
        x=x.view(-1,16*10*10)
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.fc3(x)
        
        return x
        
net=Net()
# test_data = torch.zeros((1,1,48,48))
# net(test_data)

In [14]:
# define optimizer and loss

import torch.optim as optim

loss_function=nn.CrossEntropyLoss()
optimizer=optim.SGD(net.parameters(), lr=0.001, momentum = 0.9)


In [15]:
# data from set_A

import torchvision.transforms as transforms
transform = transforms.Compose([transforms.ToTensor()])

all_images = np.load('../data/set_A/train_images.npy')[:, None]
all_labels = np.load('../data/set_A/train_labels.npy')

data = [(image, label) for image, label in zip(all_images, all_labels)]

trainloader = torch.utils.data.DataLoader(data, batch_size=4, shuffle=True, num_workers=0)

all_images = np.load('../data/set_A/test_images.npy')[:, None]
all_labels = np.load('../data/set_A/test_labels.npy')

data = [(image, label) for image, label in zip(all_images, all_labels)]
testloader = torch.utils.data.DataLoader(data, batch_size=4, shuffle=True, num_workers=0)

classes = ('pristine', 'defect')

In [18]:
# training the nn
import simulator

# if set_in()=="set_B":
#     preset_key="set_B"
#     a=0
#     print(preset_key)
# elif set_in()=="set_A":
#     preset_key="set_A"
#     a=1
#     print(preset_key)

EPOCHS=1
for epoch in range(EPOCHS):
    
    running_loss=0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels= data
        optimizer.zero_grad()
        outputs=net(inputs)
        print(i)
        #print(outputs)
        #print(inputs)
        loss=loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss+= loss.item()
        if i%200==199:
            print("[%d,%5d] loss: %.3f" % (epoch+1, i+1, running_loss/2000))
            running_loss = 0.0
print("Finished Training")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108


KeyboardInterrupt: 

In [9]:
# show the accuracy

correct=0
total=0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs= net(images)
        _, predicted=torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted==labels).sum().item()
print("Accuracy of the network: %d %%" %(100*correct/total))

Accuracy of the network: 96 %


In [8]:
# shows the accuracy of each class

class_correct=list(0. for i in range(2))
class_total=list(0. for i in range(2))
with torch.no_grad():
    for data in testloader:
        images, labels =data
        outputs= net(images)
        _, predicted=torch.max(outputs, 1)
        c= (predicted==labels).squeeze()
        for i in range(4):
            label=labels[i]
            class_correct[label]+= c[i].item()
            class_total[label]+=1

for i in range(2):
    print("Accuracy of %5s : %2d %%" %(classes[i], 100*class_correct[i]/class_total[i]))

Accuracy of pristine : 100 %
Accuracy of defect : 93 %
